In [120]:
import numpy as np
import scipy.io as sio
from scipy.signal import hilbert, coherence, welch
from scipy import signal
import pandas as pd

In [121]:
filepath = '/home/desktop/Desktop/22104412_Docs/EEG-COGMusic/EEG-Emotion/Datasets/deap/PREPROCESSED/data_preprocessed_matlab/s08.mat'
mat = sio.loadmat(filepath)

In [122]:
data = mat['data']
labels = mat['labels'][:,0:2]

In [123]:
data.shape,labels.shape

((40, 40, 8064), (40, 2))

In [124]:
# data = data.transpose((2,0,1))
# data.shape

In [125]:
# df = pd.read_excel('./participant_ratings.xls')

In [126]:
# participant_id_value = 1
# participant_df = df[df['Participant_id'] == participant_id_value]
# participant_df

In [127]:
# valence = np.array(participant_df['Valence'])
# arousal = np.array(participant_df['Arousal'])

In [128]:
# labels = np.stack((valence,arousal),1)

In [129]:
# index_reordered = np.array([1,2,3,4,7,6,8,9,20,19,21,22,23,24,25,26,32,31,29,30,28,27,17,18,16,15,13,14,12,11,5,10])-1

In [130]:
data = data[:,0:32,:]

In [131]:
data.shape

(40, 32, 8064)

In [132]:
sampling_rate = 128
data_bl = data[:,:,0:3*sampling_rate]
data = data[:,:,3*sampling_rate:]

In [133]:
l_kfold = labels

In [134]:
l_kfold

array([[7.6 , 6.78],
       [5.04, 6.51],
       [8.15, 6.96],
       [6.82, 8.23],
       [4.41, 7.72],
       [7.4 , 7.26],
       [5.04, 5.04],
       [8.82, 6.4 ],
       [8.78, 7.24],
       [4.97, 3.6 ],
       [8.41, 5.  ],
       [8.47, 6.54],
       [7.9 , 5.09],
       [6.88, 4.97],
       [8.33, 5.04],
       [4.21, 5.06],
       [6.67, 4.77],
       [7.21, 4.03],
       [5.04, 6.28],
       [6.74, 4.21],
       [4.68, 6.41],
       [4.23, 5.79],
       [3.59, 3.68],
       [4.15, 4.56],
       [3.72, 3.9 ],
       [4.99, 4.99],
       [4.46, 4.59],
       [5.04, 4.96],
       [2.78, 2.99],
       [1.4 , 3.13],
       [1.45, 6.6 ],
       [6.33, 8.23],
       [4.44, 6.44],
       [5.05, 6.83],
       [3.62, 6.67],
       [1.21, 7.94],
       [5.06, 3.31],
       [4.72, 4.69],
       [5.05, 3.46],
       [3.13, 5.03]])

In [135]:
for i in range(data.shape[0]):
    d = data[i,:,:]
    md = data_bl[i,:,:]
    m = np.mean(md,1)
    m = m[:,np.newaxis]
    data[i,:,:] = d-m

In [136]:
data.shape

(40, 32, 7680)

In [137]:
## Z score normalization
data = (data - np.mean(data,2)[:,:,np.newaxis])/np.std(data,2)[:,:,np.newaxis]
data.shape

(40, 32, 7680)

In [138]:
data_dict = {}
labels_dict = {}
for i in range(40):
    data_dict[str(i)] = data[i*1:(i+1)*1,:,:]
    labels_dict[str(i)] = labels[i*1:(i+1)*1,:]

for key,dt in data_dict.items():
    print(f'Dict Key {key} processed')
    lb = labels_dict[key]
    window_length = 3
    window_stride = 0.5
    segments = int((dt.shape[2]-window_length*sampling_rate)/(window_stride*sampling_rate) + 1)
    segmented_data_per_channel = np.zeros((dt.shape[0],dt.shape[1],segments, window_length*sampling_rate))
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            for k in range(segments):
                start = k*window_stride
                stop = window_length+start
                segmented_data_per_channel[i,j,k,:] = dt[i,j,int(start*sampling_rate):int(stop*sampling_rate)]
    segmented_data_per_channel = np.transpose(segmented_data_per_channel,(0,2,1,3))
    labels_segment = []
    dom_segment = []
    for i in range(segmented_data_per_channel.shape[0]):
        for j in range(segmented_data_per_channel.shape[1]):
            labels_segment.append(lb[i,:])
    labels_segment = np.array(labels_segment)
    labels_target = np.array(labels_segment>5, dtype=np.int8)
    labels_dict[key] = labels_target
    data_dict[key] = segmented_data_per_channel[0]

data_dict = np.array(list(data_dict.values()))
labels_dict = np.array(list(labels_dict.values()))

Dict Key 0 processed
Dict Key 1 processed
Dict Key 2 processed
Dict Key 3 processed
Dict Key 4 processed
Dict Key 5 processed
Dict Key 6 processed
Dict Key 7 processed
Dict Key 8 processed
Dict Key 9 processed
Dict Key 10 processed
Dict Key 11 processed
Dict Key 12 processed
Dict Key 13 processed
Dict Key 14 processed
Dict Key 15 processed
Dict Key 16 processed
Dict Key 17 processed
Dict Key 18 processed
Dict Key 19 processed
Dict Key 20 processed
Dict Key 21 processed
Dict Key 22 processed
Dict Key 23 processed
Dict Key 24 processed
Dict Key 25 processed
Dict Key 26 processed
Dict Key 27 processed
Dict Key 28 processed
Dict Key 29 processed
Dict Key 30 processed
Dict Key 31 processed
Dict Key 32 processed
Dict Key 33 processed
Dict Key 34 processed
Dict Key 35 processed
Dict Key 36 processed
Dict Key 37 processed
Dict Key 38 processed
Dict Key 39 processed


In [139]:
data_dict.shape

(40, 115, 32, 384)

In [140]:
labels_dict.shape

(40, 115, 2)

In [141]:
np.where(labels_dict == [0,1])

(array([ 0,  0,  0, ..., 39, 39, 39]),
 array([  0,   1,   2, ..., 113, 114, 114]),
 array([1, 1, 1, ..., 1, 0, 1]))

In [142]:
dt = data_dict
labels = labels_dict

v = []
a = []
mc = []

for fold in labels:
    valence = []
    arousal = []
    multiclass = []
    for sample in fold:
        if (sample == [0,0]).all():
            valence.append([1,0])
            arousal.append([1,0])
            multiclass.append([1,0,0,0])
        if (sample == [0,1]).all():
            valence.append([1,0])
            arousal.append([0,1])
            multiclass.append([0,1,0,0])
        if (sample == [1,0]).all():
            valence.append([0,1])
            arousal.append([1,0])
            multiclass.append([0,0,1,0])
        if (sample == [1,1]).all():
            valence.append([0,1])
            arousal.append([0,1])
            multiclass.append([0,0,0,1])
    v.append(valence)
    a.append(arousal)
    mc.append(multiclass)

In [143]:
v = np.array(v)
a = np.array(a)
mc = np.array(mc)

In [144]:
freq_bands = [(4,8),(8,12),(12,24),(24,35),(35,45)]

In [145]:
##############################################################
#Filter the signal
def filtersignal(s,f):

    # Define bandpass filter parameters
    fs = 128  # Sampling frequency
    lowcut = f[0]  # Lower cutoff frequency (Hz)
    highcut = f[1]  # Upper cutoff frequency (Hz)
    order = 4  # Filter order

    # Design a bandpass Butterworth filter
    b, a = signal.butter(order, [lowcut/(fs/2), highcut/(fs/2)], btype='band')

    return signal.filtfilt(b, a, s)

##############################################################
#Phase Lag Index
def phase_lag_index(signal1, signal2):
    pli =  []
    for f in freq_bands:
        signal1 = filtersignal(signal1,f)
        signal2 = filtersignal(signal2,f)
        # Calculate the analytic signals
        analytic_signal1 = hilbert(signal1)
        analytic_signal2 = hilbert(signal2)
        
        # Compute the phases
        phase1 = np.angle(analytic_signal1)
        phase2 = np.angle(analytic_signal2)
        
        # Calculate the phase differences
        phase_diff = phase1 - phase2
        
        # Compute the Phase Lag Index (PLI)
        pli.append(np.abs(np.mean(np.sign(phase_diff))))
    
    return pli

##################################################################
#Phase Locking Value
def phase_locking_value(signal1, signal2):
    plv =  []
    for f in freq_bands:
        signal1 = filtersignal(signal1,f)
        signal2 = filtersignal(signal2,f)
    # Calculate the analytic signals
        analytic_signal1 = hilbert(signal1)
        analytic_signal2 = hilbert(signal2)
        
        # Compute the phases
        phase1 = np.angle(analytic_signal1)
        phase2 = np.angle(analytic_signal2)
        
        # Calculate the phase differences
        phase_diff = np.exp(1j * (phase1 - phase2))
        
        # Compute the Phase Locking Value (PLV)
        plv.append(np.abs(np.mean(phase_diff)))
    
    return plv

###################################################################
#Coherence
def coherence_signals(x, y):
    freqs, Cxy = coherence(x, y, fs=128, noverlap= 64,window='hann',nperseg=128)
    Cxy_t = np.mean(Cxy[np.logical_and(freqs >= 4, freqs < 8)])
    Cxy_a = np.mean(Cxy[np.logical_and(freqs >= 8, freqs < 12)])
    Cxy_bl = np.mean(Cxy[np.logical_and(freqs >= 12, freqs < 24)])
    Cxy_bh = np.mean(Cxy[np.logical_and(freqs >= 24, freqs < 35)])
    Cxy_g = np.mean(Cxy[np.logical_and(freqs >= 35, freqs < 45)])
    Cxy_f = [Cxy_t, Cxy_a, Cxy_bl, Cxy_bh, Cxy_g]
    return Cxy_f

#######################################################################
#PSD
def power_spectral_density(x):
    freqs,Pxx = welch(x, fs=128, noverlap= 64, window='hann',nperseg=128)
    Pxx_t = np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 8)])/np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 45)])
    Pxx_a = np.mean(Pxx[np.logical_and(freqs >= 8, freqs < 12)])/np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 45)])
    Pxx_bl = np.mean(Pxx[np.logical_and(freqs >= 12, freqs < 24)])/np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 45)])
    Pxx_bh = np.mean(Pxx[np.logical_and(freqs >= 24, freqs < 35)])/np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 45)])
    Pxx_g = np.mean(Pxx[np.logical_and(freqs >= 35, freqs < 45)])/np.mean(Pxx[np.logical_and(freqs >= 4, freqs < 45)])
    Pxx_f = [Pxx_t, Pxx_a, Pxx_bl, Pxx_bh, Pxx_g]
    return Pxx_f

In [146]:
dt.shape

(40, 115, 32, 384)

In [147]:
feature_pli = np.zeros((40,115,32,32,5))
feature_coh = np.zeros((40,115,32,32,5))
feature_psd = np.zeros((40,115,32,5))
for fold in range(dt.shape[0]):
    sample = dt[fold]
    for k in range(sample.shape[0]):
        data = sample[k]
        for i in range(data.shape[0]):
            j = 0
            while j<i:
                feature_pli[fold,k,i,j,:] = coherence_signals(data[i],data[j])
                feature_coh[fold,k,i,j,:] = phase_lag_index(data[i],data[j])
                j+=1
            feature_psd[fold,k,i,:] = power_spectral_density(data[i])
    print(f'Processed {fold} trial')

Processed 0 trial
Processed 1 trial
Processed 2 trial
Processed 3 trial
Processed 4 trial
Processed 5 trial
Processed 6 trial
Processed 7 trial
Processed 8 trial
Processed 9 trial
Processed 10 trial
Processed 11 trial
Processed 12 trial
Processed 13 trial
Processed 14 trial
Processed 15 trial
Processed 16 trial
Processed 17 trial
Processed 18 trial
Processed 19 trial
Processed 20 trial
Processed 21 trial
Processed 22 trial
Processed 23 trial
Processed 24 trial
Processed 25 trial
Processed 26 trial
Processed 27 trial
Processed 28 trial
Processed 29 trial
Processed 30 trial
Processed 31 trial
Processed 32 trial
Processed 33 trial
Processed 34 trial
Processed 35 trial
Processed 36 trial
Processed 37 trial
Processed 38 trial
Processed 39 trial


In [148]:
# feature = np.zeros((40,115,32,32,3))
# for fold in range(coh.shape[0]):
#     c = coh[fold]
#     for k in range(c.shape[0]):
#         feature[fold,k,:,:,0] = coh[fold,k,:,:,0]+np.transpose(coh[fold,k,:,:,1])
#         feature[fold,k,:,:,1] = coh[fold,k,:,:,2]+np.transpose(coh[fold,k,:,:,3])
#         feature[fold,k,:,:,2] = coh[fold,k,:,:,4]+np.transpose(coh[fold,k,:,:,5])
#     print(f'Sample {fold} processed.')

In [157]:
# feature.shape

In [158]:
dt[0][0].shape

(32, 384)

In [159]:
# for fold in range(dt.shape[0]):
#     data = dt[fold]
#     for i in range(data.shape[0]):
#         signal_1 = data[i]
#         for j in range(signal_1.shape[0]):
#             sig = signal_1[j]
#             (f,Px) = signal.welch(sig,128)
#             Px = (Px - np.mean(Px))/np.std(Px)
#             feature[fold,i,j,j,0] = np.mean(Px[np.logical_and(f >= 1, f < 4)])/np.mean(Px[np.logical_and(f >= 4, f < 8)])
#             feature[fold,i,j,j,1] =  np.mean(Px[np.logical_and(f >= 8, f < 12)])/np.mean(Px[np.logical_and(f >= 12, f < 24)])
#             feature[fold,i,j,j,2] = np.mean(Px[np.logical_and(f >= 24, f < 35)])/np.mean(Px[np.logical_and(f >= 35, f < 45)])
#     print(f'Processed {fold} sample')

In [160]:
l_kfold.shape

(40, 2)

In [161]:
l_kfold

array([[7.6 , 6.78],
       [5.04, 6.51],
       [8.15, 6.96],
       [6.82, 8.23],
       [4.41, 7.72],
       [7.4 , 7.26],
       [5.04, 5.04],
       [8.82, 6.4 ],
       [8.78, 7.24],
       [4.97, 3.6 ],
       [8.41, 5.  ],
       [8.47, 6.54],
       [7.9 , 5.09],
       [6.88, 4.97],
       [8.33, 5.04],
       [4.21, 5.06],
       [6.67, 4.77],
       [7.21, 4.03],
       [5.04, 6.28],
       [6.74, 4.21],
       [4.68, 6.41],
       [4.23, 5.79],
       [3.59, 3.68],
       [4.15, 4.56],
       [3.72, 3.9 ],
       [4.99, 4.99],
       [4.46, 4.59],
       [5.04, 4.96],
       [2.78, 2.99],
       [1.4 , 3.13],
       [1.45, 6.6 ],
       [6.33, 8.23],
       [4.44, 6.44],
       [5.05, 6.83],
       [3.62, 6.67],
       [1.21, 7.94],
       [5.06, 3.31],
       [4.72, 4.69],
       [5.05, 3.46],
       [3.13, 5.03]])

In [162]:
feature_coh_clipped = feature_coh[:,40:,:,:,:]
feature_pli_clipped = feature_pli[:,40:,:,:,:]
feature_psd_clipped = feature_psd[:,40:,:,:]
dt_clipped = dt[:,40:,:,:]
v_clipped = v[:,40:,:]
a_clipped = a[:,40:,:]
mc_clipped = mc[:,40:,:]

In [163]:

feature_coh_clipped.shape, feature_pli_clipped.shape, feature_psd_clipped.shape, dt_clipped.shape,v_clipped.shape, a_clipped.shape,mc_clipped.shape

((40, 75, 32, 32, 5),
 (40, 75, 32, 32, 5),
 (40, 75, 32, 5),
 (40, 75, 32, 384),
 (40, 75, 2),
 (40, 75, 2),
 (40, 75, 4))

In [164]:
sio.savemat('./datasets/s08_datasets_Zscore_clipped.mat',{'coh' : feature_coh_clipped,
                                           'pli' : feature_pli_clipped,
                                           'psd' : feature_psd_clipped,
                                           'labels_kfold': l_kfold,
                                           'valence': v_clipped,
                                           'arousal' : a_clipped,
                                           'multiclass' : mc_clipped,
                                           'EEGNet': dt_clipped})